<a href="https://colab.research.google.com/github/arsalanyaghoobi/BertForTokenClassification-using-keras/blob/main/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name Entity recognition `build knowledge from unstructured text data`. It parses important information form the text like `email address, phone number, degree titles, location names, organizations, time and etc`.


Named Entity Recognition is the task of determining a set of entities in the sentence.
In other words, each word is classified to one of the predefined entities.
BERT models can be used to solve the NER task by adding a softmax layer after the last embedding layer. Since the embeddings generated from BERT already hold information about the realtion between the word and the other words of the sentence and thus are suitable for the NER task since the entities can hugely depend on the context

https://androidkt.com/name-entity-recognition-with-bert-in-tensorflow/

In [248]:
pip install transformers

Transfomers library contains some state-of-the-art pre-trained models for Natural Language Processing (NLP) like BERT, GPT, XLNet … etc.

In [249]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os

Pandas is the most popular data manipulation package in Python.Bear in mind that Panda is different from Pandas ; and Pandas is our case.

NumPy aims to provide an array object that is up to 50x faster that traditional Python lists. it provides a lot of supporting functions that make working with ndarray very easy. Arrays are very frequently used in data science, where speed and resources are very important.An array is a data structure that stores values of same data type. In Python, this is the main difference between arrays and lists. While python lists can contain values corresponding to different data types, arrays in python can only contain values corresponding to same data type.

In [250]:
#from google.colab import files
#files.upload()


In [251]:
df_data = pd.read_csv("/content/ner_dataset.csv", error_bad_lines=False,sep=",",encoding="latin1").fillna(method='ffill')
df_data.head

<bound method NDFrame.head of               Sentence #           Word  POS Tag
0            Sentence: 1      Thousands  NNS   O
1            Sentence: 1             of   IN   O
2            Sentence: 1  demonstrators  NNS   O
3            Sentence: 1           have  VBP   O
4            Sentence: 1        marched  VBN   O
...                  ...            ...  ...  ..
1048570  Sentence: 47959           they  PRP   O
1048571  Sentence: 47959      responded  VBD   O
1048572  Sentence: 47959             to   TO   O
1048573  Sentence: 47959            the   DT   O
1048574  Sentence: 47959         attack   NN   O

[1048575 rows x 4 columns]>

Now we split the dataset to use 20% to validate the model.

In [252]:
from sklearn.model_selection import train_test_split
x_train,x_test = train_test_split(df_data,test_size = 0.20, shuffle=False)
print(x_train)
#print(x_test)

             Sentence #           Word  POS Tag
0           Sentence: 1      Thousands  NNS   O
1           Sentence: 1             of   IN   O
2           Sentence: 1  demonstrators  NNS   O
3           Sentence: 1           have  VBP   O
4           Sentence: 1        marched  VBN   O
...                 ...            ...  ...  ..
838855  Sentence: 38346           raft   NN   O
838856  Sentence: 38346           from   IN   O
838857  Sentence: 38346            one   CD   O
838858  Sentence: 38346             of   IN   O
838859  Sentence: 38346            the   DT   O

[838860 rows x 4 columns]


Sklearn,  features various classification, regression and clustering algorithms.
Provides train/test indices to split data in train/test sets. Split dataset into k consecutive folds (without shuffling by default).

drawing stacked bar plot using 
`data.groupby('feature')['label'].value_counts()`

pandas objects can be split on any of their axes. The abstract definition of grouping is to provide a mapping of labels to group names. To create a GroupBy object.

In [253]:
agg_func = lambda s: [[w,t] for w,t in zip(s["Word"].values.tolist(),s["Tag"].values.tolist())]    #What is agg function?read the text above.
#agg_func = lambda w,t: [[w,t] for w,t in zip(["Word"].values.tolist(),["Tag"].values.tolist())]      #I changed from zip to ner_dataset and I got error in the cell
                                                                                                     #below; where is Zip?                                                    

In [254]:
x_train_grouped = x_train.groupby("Sentence #").apply(agg_func)  # make list of 838860 row* 3 column including Tag, Name and Sentences.
x_test_grouped = x_test.groupby("Sentence #").apply(agg_func)    

In [255]:
print(x_train_grouped)

Sentence #
Sentence: 1        [[Thousands, O], [of, O], [demonstrators, O], ...
Sentence: 10       [[Iranian, B-gpe], [officials, O], [say, O], [...
Sentence: 100      [[Helicopter, O], [gunships, O], [Saturday, B-...
Sentence: 1000     [[They, O], [left, O], [after, O], [a, O], [te...
Sentence: 10000    [[U.N., B-geo], [relief, O], [coordinator, O],...
                                         ...                        
Sentence: 9995     [[Opposition, O], [leader, O], [Mir, O], [Hoss...
Sentence: 9996     [[On, O], [Thursday, B-tim], [,, O], [Iranian,...
Sentence: 9997     [[Following, O], [Iran, B-geo], ['s, O], [disp...
Sentence: 9998     [[Since, O], [then, O], [,, O], [authorities, ...
Sentence: 9999     [[The, O], [United, B-org], [Nations, I-org], ...
Length: 38346, dtype: object


In [256]:
x_train_sentences = [[s[0] for s in sent] for sent in x_train_grouped.values]       #seperation of words from tags in both training and testing dataset
x_test_sentences = [[s[0] for s in sent] for sent in x_test_grouped.values]         #s[0]= Words , s[1]=Tag
print(x_train_sentences[10])                                                        #what is sent??????????

['In', 'Beirut', ',', 'a', 'string', 'of', 'officials', 'voiced', 'their', 'anger', ',', 'while', 'at', 'the', 'United', 'Nations', 'summit', 'in', 'New', 'York', ',', 'Prime', 'Minister', 'Fouad', 'Siniora', 'said', 'the', 'Lebanese', 'people', 'are', 'resolute', 'in', 'preventing', 'such', 'attempts', 'from', 'destroying', 'their', 'spirit', '.']


In [257]:
x_train_tags = [[t[1] for t in tag] for tag in x_train_grouped.values]            #seperation of words from tags in both training and testing dataset
x_test_tags = [[t[1] for t in tag] for tag in x_test_grouped.values]              #s[1]? or t[1]?
print(x_train_tags[10])

['O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-org', 'I-org', 'O', 'O', 'B-geo', 'I-geo', 'O', 'B-per', 'O', 'B-per', 'I-per', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [258]:
MAX_LENGTH = 128
BERT_MODEL = 'bert-base-cased'
BATCH_SIZE = 32
pad_token = 0
#pad_token_segment_id = 0    #??????
#sequence_a_segment_id = 0   #??????

In [259]:
from transformers import(
    #TF2_WEIGHTS_NAME,
    BertConfig,
    BertTokenizer,
    TFBertForTokenClassification,
    #create_optimizer
    )
tokenizer_class = BertTokenizer

tokenizer = tokenizer_class.from_pretrained(BERT_MODEL,do_lower_case=False)

TFBertForTokenClassification is a fine-tuning model that wraps BertModel and adds token-level classifier on top of the BertModel. We load the pre-trained “bert-base-cased” model and provide the number of possible labels.

Creat a Model:

`for any Bert model we need an input and a label`;NER is the multi-class classification problem where the words are our input and tags are our labels.

The base class PretrainedConfig `implements the common methods for loading/saving a configuration either from a local file or directory`, or from a pretrained model configuration provided by the library (downloaded from HuggingFace’s AWS S3 repository)

In [260]:
MODEL_CLASSES = {'bert':  (BertConfig, TFBertForTokenClassification, BertTokenizer)}
config_class, model_class, tokenizer_class = MODEL_CLASSES['bert']
config = config_class.from_pretrained(BERT_MODEL,num_labels=50000)  #what is the number of labels?

model =model_class.from_pretrained(
    BERT_MODEL,
    from_pt= bool('.bin'in BERT_MODEL),
    config = config
)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertForTokenClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['dropout_493', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [261]:
print(model.summary())

Model: "tf_bert_for_token_classification_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
_________________________________________________________________
dropout_493 (Dropout)        multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  38450000  
Total params: 146,760,272
Trainable params: 146,760,272
Non-trainable params: 0
_________________________________________________________________
None


The tensor y_true is the true data (or target, ground truth) you pass to the fit method.
It's a conversion of the numpy array y_train into a tensor.

1)Two very easy ways to find the shape of y_true are:
check your true/target data: print(Y_train.shape)
check your model.summary() and see the last output; if your last layer outputs (None, 1), the shape of y_true is (batch, 1). If the last layer outputs (None, 200,200, 3), then y_true will be (batch, 200,200,3).

In [262]:
model.layers[-1].activation = tf.keras.activations.softmax

Prepare Input

Before we can start fine-tuning the model, we have to prepare the data set for use with BERT.We need to set the text into 3 kinds of embeddings:

1)Token Embedding:
In order to make token embedding, we need to map the word token into the id.

2)In order to make mask word embedding, we need to use 1 to indicate the real toke and 0 to indicate to pad token.

3)Where “token_type_ids” are used to indicate whether this is the first # sequence or the second sequence.

Next, we cut and pad the token and label sequences to our desired length.





In [263]:
tag_list=df_data.Tag.unique()   #tag_list will help you to create your label_map 
print(tag_list)

label_map = {label: i for i, label in enumerate(tag_list)}      # we need to define number of nodes for the very last layers; number of nodes equal to number of out put Entity.
#It is used for defining lable_ids   

num_labels = len(tag_list) + 1    #you will use number of labels for configuraion stage.
num_labels                                                          #????????

['O' 'B-geo' 'B-gpe' 'B-per' 'I-geo' 'B-org' 'I-org' 'B-tim' 'B-art'
 'I-art' 'I-per' 'I-gpe' 'I-tim' 'B-nat' 'B-eve' 'I-eve' 'I-nat']


18

In [264]:
config_class, model_class, tokenizer_class = MODEL_CLASSES['bert']

In [265]:
config = config_class.from_pretrained(BERT_MODEL,num_labels=num_labels)   

In [266]:
model = model_class.from_pretrained(
                BERT_MODEL,
                from_pt=bool(".bin" in BERT_MODEL),
                config=config)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertForTokenClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['dropout_531', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [267]:
 model.layers[-1].activation = tf.keras.activations.softmax     #defining he very last layer activation

In [268]:
!pip install tqdm
from tqdm import tqdm


In [269]:
from keras.preprocessing.sequence import pad_sequences
max_seq_length =128

def convert_to_input(sentences,tags):
  input_id_list,attention_mask_list,token_type_id_list=[],[],[]
  label_id_list=[]   #did it differe to have this among the upper items?
  
  for x,y in tqdm(zip(sentences,tags), total=len(tags)): #?????????
  
    tokens = []
    label_ids = []

    for word, label in zip(x, y):
      word_tokens = tokenizer.tokenize(word)
      tokens.extend(word_tokens)  #Python extend() is an inbuilt function that adds the specified list 
                                  #elements (or any iterable) to the end of the current list. 
                                  #The extend() method extends the list by adding all items of the list (passed as an argument) to an end.
     # Use the real label id for the first token of the word, and padding ids for the remaining tokens
      label_ids.extend([label_map[label]] + [pad_token_label_id] * (len(word_tokens) - 1))

  
    special_tokens_count =  2
    if len(tokens) > max_seq_length - special_tokens_count:
      tokens = tokens[: (max_seq_length - special_tokens_count)]
      label_ids = label_ids[: (max_seq_length - special_tokens_count)]

    label_ids = [pad_token_label_id]+label_ids+[pad_token_label_id]
    inputs = tokenizer.encode_plus(tokens,add_special_tokens=True, max_length=max_seq_length)

    input_ids, token_type_ids = inputs["input_ids"], inputs["token_type_ids"]
    attention_masks = [1] * len(input_ids)

    attention_mask_list.append(attention_masks)
    input_id_list.append(input_ids)
    token_type_id_list.append(token_type_ids)

    label_id_list.append(label_ids)

  return input_id_list,token_type_id_list,attention_mask_list,label_id_list


In [270]:
pad_token_label_id = 0

In [271]:
input_ids_train,token_ids_train,attention_masks_train,label_ids_train=convert_to_input(x_train_sentences,x_train_tags)

100%|██████████| 38346/38346 [00:42<00:00, 905.62it/s]


In [272]:
input_ids_test,token_ids_test,attention_masks_test,label_ids_test=convert_to_input(x_test_sentences,x_test_tags)

100%|██████████| 9614/9614 [00:10<00:00, 932.42it/s]


In [273]:
input_ids_train = pad_sequences(input_ids_train,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
token_ids_train = pad_sequences(token_ids_train,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
attention_masks_train = pad_sequences(attention_masks_train,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
label_ids_train = pad_sequences(label_ids_train,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")

In [274]:
input_ids_test = pad_sequences(input_ids_test,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
token_ids_test = pad_sequences(token_ids_test,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
attention_masks_test = pad_sequences(attention_masks_test,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
label_ids_test = pad_sequences(label_ids_test,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")

In [275]:
np.shape(input_ids_train),np.shape(token_ids_train),np.shape(attention_masks_train),np.shape(label_ids_train),

((38346, 128), (38346, 128), (38346, 128), (38346, 128))

In [276]:
np.shape(input_ids_test),np.shape(token_ids_test),np.shape(attention_masks_test),np.shape(label_ids_test),

((9614, 128), (9614, 128), (9614, 128), (9614, 128))

In [277]:
def example_to_features(input_ids,attention_masks,token_type_ids,y):
  return {"input_ids": input_ids,
          "attention_mask": attention_masks,
          "token_type_ids": token_type_ids},y

train_ds = tf.data.Dataset.from_tensor_slices((input_ids_train,attention_masks_train,token_ids_train,label_ids_train)).map(example_to_features).shuffle(1000).batch(32).repeat(5)


test_ds=tf.data.Dataset.from_tensor_slices((input_ids_test,attention_masks_test,token_ids_test,label_ids_test)).map(example_to_features).batch(1)


In [278]:
from keras.preprocessing.sequence import pad_sequences
max_seq_length =128

In [279]:
model.summary()

Model: "tf_bert_for_token_classification_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
_________________________________________________________________
dropout_531 (Dropout)        multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  13842     
Total params: 108,324,114
Trainable params: 108,324,114
Non-trainable params: 0
_________________________________________________________________


In [280]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')


In [281]:
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
history = model.fit(train_ds, epochs=3, validation_data=test_ds)